In [1]:
from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKeyConstraint, CheckConstraint
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime
from sqlalchemy import DateTime
from sqlalchemy import ForeignKey, Boolean
from sqlalchemy.orm import relationship, backref

In [2]:
Base = declarative_base()


class Cookie(Base):
    __tablename__ = 'cookies'

    cookie_id = Column(Integer(), primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))

    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}'), " \
                       "cookie_recipe_url='{self.cookie_recipe_url}', " \
                       "cookie_sku='{self.cookie_sku}', " \
                       "quantity={self.quantity}, " \
                       "unit_cost={self.unit_cost})".format(self=self)


class User(Base):
    __tablename__ = 'users'

    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return "User(username='{self.username}', " \
                    "email_address='{self.email_address}', " \
                    "phone='{self.phone}', " \
                    "password='{self.password}')".format(self=self)


class Order(Base):
    __tablename__ = 'orders'

    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    shipped = Column(Boolean(), default=False)
    user = relationship('User', backref=backref('orders', order_by=order_id))

    def __repr__(self):
        return "Order(user_id={self.user_id}, " \
                    "shipped={self.shipped})".format(self=self)


class LineItem(Base):
    __tablename__ = 'line_items'

    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))

    order = relationship('Order', backref=backref('line_items', order_by=line_item_id))
    cookie = relationship('Cookie', uselist=False)

    def __repr__(self):
        return "LineItems(order_id={self.order_id}, "\
                        "cookie_id={self.cookie_id}, " \
                        "quantity={self.quantity}, " \
                        "extended_cost={self.extended_cost})".format(self=self)

# Persisting the Schema

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///cookies.db')

Base.metadata.create_all(engine)

# The Session

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

# Inserting Data

In [5]:
cc_cookie = Cookie(cookie_name='chocolate chip',
                   cookie_recipe_url='http://some.aweso.me/cookie/recipe.html',
                   cookie_sku='CC01',
                   quantity=12,
                   unit_cost=0.50)
session.add(cc_cookie)
session.commit()

In [6]:
cx_cookie = Cookie(cookie_name='chocolate chip test',
                   cookie_recipe_url='http://some.aweso.me/cookie/test_recipe.html',
                   cookie_sku='CC01',
                   quantity=12,
                   unit_cost=0.50)
session.add(cx_cookie)

In [7]:
session.commit()

## Multiple inserts

In [8]:
dcc = Cookie(cookie_name='dark chocolate chip',
            cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
            cookie_sku='CC02',
            quantity=1,
            unit_cost=0.75)
mol = Cookie(cookie_name='molasses',
            cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html',
            cookie_sku='MOL01',
            quantity=1,
            unit_cost=0.80)
session.add(dcc)
session.add(mol)
session.flush()
print(dcc.cookie_id)
print(mol.cookie_id)

11
12


In [9]:
c1 = Cookie(cookie_name='peanut butter',
            cookie_recipe_url='http://some.aweso.me/cookie/peanut.html',
            cookie_sku='PB01',
            quantity=24,
            unit_cost=0.25)
c2 = Cookie(cookie_name='oatmeal raisin',
            cookie_recipe_url='http://some.okay.me/cookie/raisin.html',
            cookie_sku='EWW01',
            quantity=100,
            unit_cost=1.00)
session.bulk_save_objects([c1,c2])
session.commit()
print(c1.cookie_id)

None


# Querying Data

In [10]:
cookies = session.query(Cookie).all()
print(cookies)

[Cookie(cookie_name='peanut_butter'), cookie_recipe_url='https://some.oke.me/cookies/rsia.html', cookie_sku='PON01', quantity=24, unit_cost=0.25), Cookie(cookie_name='oatmeal raisin'), cookie_recipe_url='http://some.okay.me/cookie/raisin.html', cookie_sku='EWW01', quantity=100, unit_cost=1.00), Cookie(cookie_name='chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=112, unit_cost=0.50), Cookie(cookie_name='molasses'), cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.80), Cookie(cookie_name='peanut butter'), cookie_recipe_url='http://some.aweso.me/cookie/peanut.html', cookie_sku='PB01', quantity=24, unit_cost=0.25), Cookie(cookie_name='oatmeal raisin'), cookie_recipe_url='http://some.okay.me/cookie/raisin.html', cookie_sku='EWW01', quantity=100, unit_cost=1.00), Cookie(cookie_name='chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sk

In [11]:
for cookie in session.query(Cookie):
    print(cookie)

Cookie(cookie_name='peanut_butter'), cookie_recipe_url='https://some.oke.me/cookies/rsia.html', cookie_sku='PON01', quantity=24, unit_cost=0.25)
Cookie(cookie_name='oatmeal raisin'), cookie_recipe_url='http://some.okay.me/cookie/raisin.html', cookie_sku='EWW01', quantity=100, unit_cost=1.00)
Cookie(cookie_name='chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=112, unit_cost=0.50)
Cookie(cookie_name='molasses'), cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.80)
Cookie(cookie_name='peanut butter'), cookie_recipe_url='http://some.aweso.me/cookie/peanut.html', cookie_sku='PB01', quantity=24, unit_cost=0.25)
Cookie(cookie_name='oatmeal raisin'), cookie_recipe_url='http://some.okay.me/cookie/raisin.html', cookie_sku='EWW01', quantity=100, unit_cost=1.00)
Cookie(cookie_name='chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01

## Ordering

In [12]:
from sqlalchemy import desc
from sqlalchemy import func
for cookie in session.query(Cookie).order_by(desc(Cookie.quantity)).limit(2):
    print('{:3} - {}'.format(cookie.quantity, cookie.cookie_name))


112 - chocolate chip
100 - oatmeal raisin


In [13]:
inv_count = session.query(func.sum(Cookie.quantity)).scalar()
print(inv_count)

511


In [14]:
rec_count = session.query(func.count(Cookie.cookie_name)).first()
print(rec_count)

(12,)


In [15]:
rec_count = session.query(func.count(Cookie.cookie_name).label('inventory_count')).first()
print(rec_count.keys())
print(rec_count.inventory_count)

['inventory_count']
12


## Filtering

In [16]:
record = session.query(Cookie).filter(Cookie.cookie_name == 'chocolate chip').first()
print(record)

Cookie(cookie_name='chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=112, unit_cost=0.50)


In [17]:
record = session.query(Cookie).filter_by(cookie_name='chocolate chip').first()
print(record)

Cookie(cookie_name='chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=112, unit_cost=0.50)


In [18]:
query = session.query(Cookie).filter(Cookie.cookie_name.like('%chocolate%'))
for record in query:
    print(record.cookie_name)

chocolate chip
chocolate chip
chocolate chip test
dark chocolate chip


In [19]:
from sqlalchemy import cast
query = session.query(Cookie.cookie_name, 
        cast((Cookie.quantity * Cookie.unit_cost),
              Numeric(12, 2)).label('inv_cost'))

for result in query:
    print(f'{result.cookie_name} - {result.inv_cost}')

peanut_butter - 6.00
oatmeal raisin - 100.00
chocolate chip - 56.00
molasses - 0.80
peanut butter - 6.00
oatmeal raisin - 100.00
chocolate chip - 6.00
chocolate chip test - 6.00
dark chocolate chip - 0.75
molasses - 0.80
peanut butter - 6.00
oatmeal raisin - 100.00


## Conjunctions

In [20]:
query = session.query(Cookie).filter(
    Cookie.quantity > 23,
    Cookie.unit_cost < 0.40
)
for result in query:
    print(result.cookie_name)

peanut_butter
peanut butter
peanut butter


In [21]:
from sqlalchemy import and_, not_, or_ 
query = session.query(Cookie).filter(
    or_(
        Cookie.quantity.between(10, 50),
        Cookie.cookie_name.contains('chip')
    )
)
for result in query:
    print(result.cookie_name)

peanut_butter
chocolate chip
peanut butter
chocolate chip
chocolate chip test
dark chocolate chip
peanut butter


# Updating Data

In [22]:
query = session.query(Cookie)
cc_cookie = query.filter(Cookie.cookie_name == 'chocolate chip').first()
cc_cookie.quantity = cc_cookie.quantity + 120
session.commit()
print(cc_cookie.quantity)

232


In [23]:
query = session.query(Cookie)
query = query.filter(Cookie.cookie_name == 'chocolate chip')
query.update({Cookie.quantity: Cookie.quantity - 20})
cc_cookie = query.first()
print(cc_cookie.quantity)

212


# Deleting Data

In [24]:
query = session.query(Cookie)
query = query.filter(Cookie.cookie_name == 'dark chocolate chip')
dcc_cookie = query.one()
session.delete(dcc_cookie)
session.commit()
print(dcc_cookie)

Cookie(cookie_name='dark chocolate chip'), cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75)


In [25]:
query = session.query(Cookie)
query = query.filter(Cookie.cookie_name == 'molasses')
query.delete()
mol_cookie = query.first()
print(mol_cookie)

None


# Adding related objects

In [26]:
cookiemon = User(username='cookiemon',
                email_address='mon@cookie.com',
                phone='111-111-1111',
                password='password')
cakeeater = User(username='cakeeater',
                email_address='cakeeater@cake.com',
                phone='222-222-2222',
                password='password')
pieperson = User(username='pieperson',
                email_address='person@pie.com',
                phone='333-333-3333',
                password='password')
session.add(cookiemon)
session.add(cakeeater)
session.add(pieperson)
session.commit()

o1 = Order()
o1.user = cookiemon
session.add(o1)

cc = session.query(Cookie).filter(Cookie.cookie_name=='chocolate chip').one()
line1 = LineItem(cookie=cc, quantity=2, extended_cost=1.0)
pd = session.query(Cookie).filter(Cookie.cookie_name=="peanut butter").one()
line2 = LineItem(quantity=12, extended_cost=3.00)
line2.cookie = pd
line2.order = o1

o1.line_items.append(line1)
o1.line_items.append(line2)

session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: users.username
[SQL: INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2021-03-17 14:45:36.161704', '2021-03-17 14:45:36.161725')]
(Background on this error at: http://sqlalche.me/e/13/gkpj)